In [6]:
import fastf1
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# --- CONFIGURAZIONE ---
YEAR = 2025
RACE = 'Bahrain'
SESSION = 'R'  # R = Gara
SPATIAL_RESOLUTION = 10  # Metri (Risoluzione della griglia)
SELECTED_DRIVERS = ['VER', 'PER', 'SAI', 'LEC', 'RUS', 'NOR', 'HAM', 'PIA', 'ALO', 'STR'] # Top 10
# Nota: In Bahrain 2024 le Medium sono state usate pochissimo.
# Per semplificare D-STEM, teniamo solo SOFT e HARD.
KEEP_COMPOUNDS = ['SOFT', 'HARD'] 

# Abilita la cache (crea una cartella 'f1_cache' nel tuo pc)
fastf1.Cache.enable_cache('f1_cache') 

print(f"Caricamento {RACE} {YEAR}...")
session = fastf1.get_session(YEAR, RACE, SESSION)
session.load()

# --- 1. PREPARAZIONE DATI ---
laps = session.laps

# Filtri fondamentali per Statistics for High Dimensional Data
# Vogliamo solo i giri lanciati "rappresentativi"
laps = laps.pick_drivers(SELECTED_DRIVERS)
laps = laps.pick_quicklaps()  # Rimuove giri troppo lenti
laps = laps[laps['PitInTime'].isnull()] # Rimuove giri di rientro ai box
laps = laps[laps['PitOutTime'].isnull()] # Rimuove giri di uscita dai box

# Filtro Safety Car / VSC
# TrackStatus '1' significa 'Green Flag' (Pista libera)
# Se il TrackStatus contiene numeri diversi da '1', lo scartiamo
laps = laps[laps['TrackStatus'] == '1']

# Filtro Mescole (Teniamo solo S e H per un confronto binario pulito)
laps = laps[laps['Compound'].isin(KEEP_COMPOUNDS)]

print(f"Numero di giri validi dopo il filtraggio: {len(laps)}")

# --- 2. CREAZIONE GRIGLIA SPAZIALE (RESAMPLING) ---
# Il circuito del Bahrain è lungo circa 5412m. 
# Creiamo un vettore distanza fisso: 0, 10, 20, ..., 5410
circuit_length = session.get_circuit_info().corners.iloc[-1]['Distance'] # Lunghezza approssimativa o max distance
if pd.isna(circuit_length): # Fallback se info mancante
     circuit_length = 5412 
     
spatial_grid = np.arange(0, int(circuit_length), SPATIAL_RESOLUTION)
n_points = len(spatial_grid)

print(f"Griglia spaziale creata: {n_points} punti (ogni {SPATIAL_RESOLUTION}m)")

# Liste per raccogliere i dati
all_data = []

# --- 3. LOOP DI INTERPOLAZIONE ---
print("Inizio interpolazione telemetria (potrebbe richiedere qualche minuto)...")

for i, lap in laps.iterrows():
    try:
        # Ottieni telemetria del singolo giro
        tel = lap.get_telemetry()
        
        # Se mancano dati nel giro, saltalo
        if len(tel) < 100: continue
            
        # Dati grezzi del giro
        dist_raw = tel['Distance'].values
        speed_raw = tel['Speed'].values
        x_raw = tel['X'].values
        y_raw = tel['Y'].values
        
        # --- INTERPOLAZIONE ---
        # Spalma la velocità e le coordinate X,Y sulla nostra griglia fissa
        # fill_value='extrapolate' serve per coprire i primi/ultimi metri se mancano
        f_speed = interp1d(dist_raw, speed_raw, kind='linear', fill_value="extrapolate")
        f_x = interp1d(dist_raw, x_raw, kind='linear', fill_value="extrapolate")
        f_y = interp1d(dist_raw, y_raw, kind='linear', fill_value="extrapolate")
        
        speed_interp = f_speed(spatial_grid)
        x_interp = f_x(spatial_grid)
        y_interp = f_y(spatial_grid)
        
        # Raccogli i metadati (Covariate per D-STEM)
        driver = lap['Driver']
        lap_num = lap['LapNumber']
        tyre_age = lap['TyreLife']
        compound = lap['Compound']
        
        # Per ogni punto della griglia spaziale, crea una riga
        # Nota: Per Matlab spesso è meglio un formato "Lungo" (Long Format)
        # che poi si trasforma in matrice.
        for j, s_point in enumerate(spatial_grid):
            all_data.append({
                'Driver': driver,
                'LapNumber': lap_num,
                'Space_Distance': s_point, # Coordinata 1D (s) per D-STEM
                'X_Coord': x_interp[j],    # Per plot mappa
                'Y_Coord': y_interp[j],    # Per plot mappa
                'Speed': speed_interp[j],  # La tua Y
                'TyreAge': tyre_age,       # Covariata X1
                'Compound': compound,      # Covariata X2
                'Is_Soft': 1 if compound == 'SOFT' else 0 # Dummy Variable pronta
            })
            
    except Exception as e:
        print(f"Errore nel giro {lap['LapNumber']} di {lap['Driver']}: {e}")

# --- 4. SALVATAGGIO CSV ---
df_final = pd.DataFrame(all_data)

# Controllo veloce
print(df_final.head())
print(f"Dataset finale generato con {len(df_final)} osservazioni.")

filename = f"f1_data_dstem_{RACE}_{YEAR}.csv"
df_final.to_csv(filename, index=False)
print(f"Salvato come: {filename}")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

Caricamento Bahrain 2025...


_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
core        WARNING 	Driver 63: Lap timing integrity check failed for 3 lap(s)
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
_api           INFO 	Parsing car data...
_api        WARNING 	Driver  2: Car data is incomplete!
_api        WARNING 	Driver  3: Car data is incomplete!
_api        WARNING 	Driver  8: Car data is incomplete!
_api        WARNING 	Driver  9: Car data is incomplete!
_api        WARNING 	Driver 11: Car data is incomplete!
_api        WARNING 	Driver 15: Car data is incomplete!
_api        WARNING 	Driver 17: Car data is incomplete!
_api        WARNING 	Driver 19: Car data is incom

Numero di giri validi dopo il filtraggio: 194
Griglia spaziale creata: 497 punti (ogni 10m)
Inizio interpolazione telemetria (potrebbe richiedere qualche minuto)...
  Driver  LapNumber  Space_Distance     X_Coord      Y_Coord      Speed  \
0    PIA        1.0               0 -261.998204  3554.004737   0.000000   
1    PIA        1.0              10 -258.307259  3638.116670  41.663253   
2    PIA        1.0              20 -254.275949  3738.658198  67.267760   
3    PIA        1.0              30 -249.500125  3847.915376  84.094416   
4    PIA        1.0              40 -246.010621  3927.113841  98.407309   

   TyreAge Compound  Is_Soft  
0      4.0     SOFT        1  
1      4.0     SOFT        1  
2      4.0     SOFT        1  
3      4.0     SOFT        1  
4      4.0     SOFT        1  
Dataset finale generato con 96418 osservazioni.
Salvato come: f1_data_dstem_Bahrain_2025.csv
